### Abhängigkeiten Importieren

In [ ]:
import ipycanvas
import ipywidgets
import numpy
import math

### Globale Konstanten

In [ ]:
BOARD_SIZE = 8

BLACK = -1 # MINIMIZING PLAYER
NONE = 0
WHITE = 1 # MAXIMIZING PLAYER

SHOW_FRONTIER = False
SHOW_POSSIBLE_MOVES = True

### Game State Klasse

Repäsentiert den Spielzustand

In [ ]:
class GameState:
    def __init__(self):
        self.board = numpy.zeros((BOARD_SIZE, BOARD_SIZE), dtype=numpy.int8)
        self.board[3, 3] = WHITE
        self.board[3, 4] = BLACK
        self.board[4, 3] = BLACK
        self.board[4, 4] = WHITE
        self.frontier = {(2,2),(2,3),(2,4),(2,5),
                         (3,2),(3,5),(4,2),(4,5),
                         (5,2),(5,3),(5,4),(5,5)}
        self.turn = BLACK
        self.num_pieces = 4
        self.game_over = False
        self.possible_moves = {(2, 3), (3, 2), (4, 5), (5, 4)}

In [ ]:
directions = {(-1,-1),(0,-1),(1,-1),(-1,0),(1,0),(-1,1),(0,1),(1,1)}

In [ ]:
class InvalidMoveException(Exception):
    pass

In [ ]:
def make_move(state, row, col):
    if (row, col) not in state.frontier:
        print(row, ", ", col)
        raise InvalidMoveException
    next_turn = state.turn
        
    possible_directions = adjacent_opposite_color_directions(state, row, col, state.turn)
    for (row_dir, col_dir) in possible_directions:
        if is_valid_directional_move(state, row, col, row_dir, col_dir, state.turn):
            next_turn = -state.turn
            convert_adjacent_cells_in_direction(state, row, col, row_dir, col_dir, state.turn)

    if next_turn != state.turn:
        state.num_pieces += 1
        state.board[(row, col)] = state.turn
        update_frontier(state, row, col)
        if can_move(state, next_turn):
            state.turn = next_turn
        elif not can_move(state, state.turn):
                state.game_over = True
                return state
        state.possible_moves = get_possible_moves(state, state.turn)
    else:
        raise InvalidMoveException()
    return state

In [ ]:
def is_move_valid(state, row, col, turn):
    dirs = [(-1,-1),(0,-1),(1,-1),(-1,0),(1,0),(-1,1),(0,1),(1,1)]
    for rowdelta, coldelta in dirs:
        try:
            if state.board[row + rowdelta, col + coldelta] == -turn and is_valid_directional_move(state, row, col, rowdelta, coldelta, turn):
                return True
        except IndexError:
            continue
    return False

In [ ]:
def get_winner(state):
    black_disks = count_disks(state, BLACK)
    white_disks = count_disks(state, WHITE)
    if black_disks > white_disks:
        return BLACK
    if white_disks > black_disks:
        return WHITE
    else:
        return NONE

In [ ]:
def get_possible_moves(state, player):
    possible_moves = []
    for (row, col) in state.frontier:
            if is_move_valid(state, row, col, player):
                possible_moves.append((row, col))
    return possible_moves

In [ ]:
def can_move(state, player):
    return len(get_possible_moves(state, player)) != 0

Looks up to a possible of 8 directions surrounding the given move. If any of the
move's surrounding cells is the opposite color of the move itself, then record
the direction it is in and store it in a list of tuples [(rowdelta, coldelta)].
Return the list of the directions at the end.

In [ ]:
def adjacent_opposite_color_directions(state, row, col, turn):
    dir_list = []
    for rowdelta in range(-1, 2):
        if not 0 <= row+rowdelta < 8:
            continue
        for coldelta in range(-1, 2):
            if not 0 <= col+coldelta < 8:
                continue
            if state.board[row + rowdelta, col + coldelta] == -turn:
                dir_list.append((rowdelta, coldelta))
    return dir_list

Given a move at specified row/col, checks in the given direction to see if
a valid move can be made. Returns True if it can; False otherwise.
Only supposed to be used in conjunction with _adjacent_opposite_color_directions()

In [ ]:
def is_valid_directional_move(state, row, col, rowdelta, coldelta, player):
        current_row = row + rowdelta
        current_col = col + coldelta

        last_cell_color = -player

        while True:
            if not (0 <= current_row < 8 and 0 <= current_col < 8):
                break
            if state.board[current_row, current_col] == NONE:
                break           
            if state.board[current_row, current_col] == player:
                last_cell_color = player
                break

            current_row += rowdelta
            current_col += coldelta
            
        return last_cell_color == player

In [ ]:
def count_disks(state, player):
    return numpy.count_nonzero(state.board == player)

In [ ]:
def get_player_string(player):
    return {BLACK: 'Black', WHITE: 'White', NONE: 'Nobody'}[player]

In [ ]:
def convert_adjacent_cells_in_direction(state, row, col, rowdelta, coldelta, player):
    ''' If it can, converts all the adjacent/contiguous cells on a turn in
        a given direction until it finally reaches the specified cell's original color '''
    if is_valid_directional_move(state, row, col, rowdelta, coldelta, player):
        current_row = row + rowdelta
        current_col = col + coldelta
        
        while state.board[current_row, current_col] == -player:
            state.board[(current_row, current_col)] = player
            current_row += rowdelta
            current_col += coldelta

In [ ]:
def update_frontier(state, row, col):
        for current_row in range(row-1, row+2):
            if not 0 <= current_row < 8:
                continue
            for current_col in range(col-1, col+2):
                if not 0 <= current_col < 8:
                    continue
                if state.board[current_row, current_col] == NONE:
                    state.frontier.add((current_row, current_col))
        state.frontier.remove((row, col))

### Widgets Initialisieren

#### Canvas Widget Initialisieren

In [ ]:
CELL_SIZE = 70

CANVAS_SIZE = BOARD_SIZE * CELL_SIZE

canvas = ipycanvas.MultiCanvas(2, width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas[0].fill_style = 'darkgreen'
canvas[0].stroke_style = 'black'
canvas[0].fill_rect(0, 0, CANVAS_SIZE, CANVAS_SIZE)
canvas[0].begin_path()
for i in range(BOARD_SIZE+1):
    pos = i * CELL_SIZE
    canvas[0].move_to(pos, 0)
    canvas[0].line_to(pos, CANVAS_SIZE)
    canvas[0].move_to(0, pos)
    canvas[0].line_to(CANVAS_SIZE, pos)
canvas[0].stroke()

#### Text Widgets initialisieren

In [ ]:
score_lbl = ipywidgets.widgets.Label()

In [ ]:
turn_lbl = ipywidgets.widgets.Label()

#### Output Widget initalisieren

In [ ]:
output = ipywidgets.widgets.Output()

In [ ]:
def display_board(state):
    update_output(state)
    display(canvas)
    display(score_lbl)
    display(turn_lbl)
    display(output)

In [ ]:
def update_output(state):
    with ipycanvas.hold_canvas(canvas):
        canvas[1].clear()
        for ((x, y), val) in numpy.ndenumerate(state.board):
            if val == NONE:
                continue
            elif val == BLACK:
                canvas[1].fill_style = 'black'
            else:
                canvas[1].fill_style = 'white'
            canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 2.2, 0, 2 * math.pi)
            
        if SHOW_FRONTIER:
            for (x, y) in state.frontier:
                canvas[1].fill_style = 'gray'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)
        
        if SHOW_POSSIBLE_MOVES:
            for (x, y) in get_possible_moves(state, state.turn):
                if state.turn == BLACK:
                    canvas[1].fill_style = 'black'
                else:
                    canvas[1].fill_style = 'white'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)
            
    score_lbl.value = f'Black Player : {count_disks(state, BLACK)} White Player : {count_disks(state, WHITE)}'
    if state.game_over:
        turn_lbl.value = f'{get_player_string(get_winner(state))} wins'
    else:
        turn_lbl.value = f'{get_player_string(state.turn)}s Move'

In [ ]:
def mouse_down(x_px, y_px):
    if not state.game_over:
        with output:
            x = math.floor(x_px / CELL_SIZE)
            y = math.floor(y_px / CELL_SIZE)
            try:
                make_move(state, x, y)
            except InvalidMoveException:
                print('Invalid Move')
            update_output(state)
            next_move(state)

canvas[1].on_mouse_down(mouse_down)

In [ ]:
state = GameState()

In [ ]:
#display_board(state)